In [104]:
# Dependencies
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import datetime
import calendar
import re

In [105]:
# Create a browser object for use by all the scrapes.
# Need to do "brew install geckodriver" to use Firefox on Mac.
# Chrome doesn't correctly show the counters on Mac.
def init_browser():
    return Browser("firefox")

### Scrape https://map.webex.com/ and collect the counters for Jan, Feb, and Mar.


In [106]:
def scrape_webex():
    browser = init_browser()

    # Visit https://map.webex.com/
    url = "https://map.webex.com/"
    browser.visit(url)

    # Give time for dynamic content to load
    time.sleep(5)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    month_ids = ['last2Month', 'lastMonth', 'nowMonth']
    months = []
    hosts_commas = []
    hosts = []
    participants_commas = []
    participants = []
    countries = []
    meetings_commas = []
    meetings = []
    minutes = []

    for month_id in month_ids:
        with browser.get_iframe('meetingMap') as iframe:
            iframe.click_link_by_id(month_id)
            time.sleep(5)
            iframe_html = iframe.html
            iframe_soup = bs(iframe_html, "html.parser")
            months.append(iframe_soup.find('span', id=month_id).get_text())
            #print(f'iframe_soup: {iframe_soup}')
            #hosts.append(iframe_soup.find('span', id='hostData'))
            hosts_commas.append(re.search('title=\"(.*)\"', str(iframe_soup.find('span', id='hostData'))).group(1))
            #hosts = ''.join(map(str, hosts))
            #hosts = hosts.replace(",", "")

            participants_commas.append(re.search('title=\"(.*)\"', str(iframe_soup.find('span', id='participantData'))).group(1))
            countries.append(re.search('title=\"(.*)\"', str(iframe_soup.find('span', id='countryData'))).group(1))
            meetings_commas.append(re.search('title=\"(.*)\"', str(iframe_soup.find('span', id='meetingData'))).group(1))
            
            mins = []
            num = 11
            while num > 0:
                a = str(iframe_soup.find('div', id='num' + str(num)))
                b = re.search(r'<span>(\d*)', a).group(1)             
                mins.append(b)
                num = num - 1
            mins_string = ''.join(map(str, mins))
            minutes.append(mins_string)
           
        
                
    hosts =[s.replace(',', '') for s in hosts_commas]
    participants = [s.replace(',', '') for s in participants_commas]
    meetings = [s.replace(',', '') for s in meetings_commas]
    
    webex = {
        "months": months,
        "hosts": hosts,
        "participants": participants,
        "countries": countries,
        "meetings": meetings,
        "minutes": minutes
    }
    

    # Close the browser after scraping
    browser.quit()

    # Return results
    return webex

In [109]:
webex_days = scrape_webex()
print(f'months: {webex_days["months"]}')
print(f'hosts: {webex_days["hosts"]}')
print(f'participants: {webex_days["participants"]}')
print(f'countries: {webex_days["countries"]}')
print(f'meetings: {webex_days["meetings"]}')
print(f'minutes: {webex_days["minutes"]}')

months: ['January 2020', 'February 2020', 'Current']
hosts: ['24504686', '24420815', '26000472']
participants: ['152398517', '161764828', '271921085']
countries: ['223', '224', '221']
meetings: ['36892704', '37994030', '58415256']
minutes: ['6310089577', '6787001490', '11012414536']


In [110]:
def get_hours(month):
    """
    Calculates the hours in a month.
    Usage: get_hours("January 2020") or get_hours("Current")
    """
    months = ["January", "February", "March", "April", "May", "June",
            "July", "August", "September", "October", "November", "December"]
    month_list = re.split(r'\s', month)
    month = month_list[0]
    if month == "Current":
        year = datetime.datetime.today().strftime("%Y")
        days = int(datetime.datetime.today().strftime("%d"))
    else:
        year = month_list[1]
        days = calendar.monthrange(int(year),int(months.index(month))+1)[1]
    hours = days * 24
    return hours


In [111]:
hours = []
for i in webex_days["months"]:
    hours_month = get_hours(i)
    hours.append(int(hours_month))

for i in range(3):
    webex["hosts"][i] = round(int(webex_days["hosts"][i]) / hours[i])
    webex["participants"][i] = round(int(webex_days["participants"][i]) / hours[i])
    webex["meetings"][i] = round(int(webex_days["meetings"][i]) / hours[i])
    webex["minutes"][i] = round(int(webex_days["minutes"][i]) / hours[i])
    
print(f'months: {webex["months"]}')
print(f'hosts: {webex["hosts"]}')
print(f'participants: {webex["participants"]}')
print(f'countries: {webex["countries"]}')
print(f'meetings: {webex["meetings"]}')
print(f'minutes: {webex["minutes"]}')
    

744
696
624
months: ['January 2020', 'February 2020', 'Current']
hosts: [32936, 35087, 41667]
participants: [204837, 232421, 435771]
countries: ['223', '224', '221']
meetings: [49587, 54589, 93614]
minutes: [8481303, 9751439, 17648100]
